In [1]:
import yfinance as yf
import investpy as inv
import pandas as pd
import datetime
import boto3
from io import StringIO

In [2]:
data_atual = datetime.datetime.now()
dt_consulta = (data_atual - datetime.timedelta(days=3)).strftime('%Y-%m-%d')

In [3]:
br_stocks = inv.stocks.get_stocks(country='brazil')

In [4]:
carteira = []

for i in br_stocks['symbol']:
    carteira.append(i+'.SA')

In [ ]:
df = yf.download(carteira, start = dt_consulta, end= dt_consulta)
df_adj_close = df['Adj Close']

df_pivot = df_adj_close.reset_index().melt(id_vars=['Date'], var_name='Stock', value_name='Price')

In [6]:
df_pivot

,Date,Stock,Price


In [7]:
# Configurações do AWS S3
bucket_name = 'nome-do-bucket'
file_name = 'stock_brazil.csv'
access_key = 'access_key'
secret_key = 'secret_key'
session_token = 'session_token'

# Criando a sessão do boto3 com as credenciais
session = boto3.Session(
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token
)

# Criando um cliente para o S3
s3 = session.client('s3')

# Convertendo o DataFrame para CSV usando StringIO
csv_buffer = StringIO()
df_pivot.to_csv(csv_buffer, index=False)

# Salvando o CSV no bucket S3
s3.put_object(Bucket=bucket_name, Key=file_name, Body=csv_buffer.getvalue())

print(f"O arquivo '{file_name}' foi salvo com sucesso no bucket '{bucket_name}'")

O arquivo 'stock_brazil.csv' foi salvo com sucesso no bucket 'tech-challenge-big-data'
